In [6]:
import urllib.request, urllib.error, urllib.parse
import json
import pandas as pd
import seaborn as sns
import plotly.express as px

## 1.Получение дневных свечей для указанного тикера

In [7]:
def get_stock_candles(ticker):
    
    # Чтобы получить DataFrame с дневными свечами достаточно вызвать функцию get_stock_candles(ticker), где ticker - тикер акции.
    # В рамках задания у обоих торгуемых инструментов одинаковые engine, market и board, поэтому они неизменны.
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'}
    df_combined = pd.DataFrame(columns = ['open', 'close', 'high', 'low', 'value', 'volume', 'date', 'end']).set_index('date')
    
    for i in range(0, 5):
        query_string = f'http://iss.moex.com/iss/engines/stock/markets/shares/boards/tqbr/securities/{ticker}/candles.json?from=2013-01-01&interval=24&start={(i)*500}'
        
        with urllib.request.urlopen(query_string) as url:
            data = json.loads(url.read().decode())['candles']['data']
            
        df = pd.DataFrame(data, columns = ['open', 'close', 'high', 'low', 'value', 'volume', 'date', 'end']).set_index('date')
        df_combined = pd.concat([df_combined, df])
   
    return df_combined

## 2.Расчет SMA 200, SMA 50 и RSI

In [13]:
def indicators(ticker):
    df = get_stock_candles(ticker)
    df['SMA50'] = df['close'].rolling(50, center = True).mean()
    df['SMA200'] = df['close'].rolling(200, center = True).mean()
    
    chg = df['close'].diff(1) / df['close'] * 100

    gain = chg.mask(chg < 0, 0)
    df['gain'] = gain

    loss = chg.mask(chg > 0, 0)
    df['loss'] = loss * (-1)
    
    df['avg_gain'] = df['gain'].rolling(14).mean()
    df['avg_loss'] = df['loss'].rolling(14).mean()
    df['RSI'] = 100 - 100 / (1+ (df['avg_gain'] / df['avg_loss'] ))
    df_modified = df
    
    return df_modified

## 3. Построение графика SMA 200, SMA 50 и RSI

In [14]:
def draw_plots(ticker):
    df = indicators(ticker)
    fig = px.line(df, x = df.index, y= [df.SMA50, df.SMA200, df.RSI])
    return fig

## 4. Расчет корреляции

In [10]:
def correlation(ticker):
    df = indicators(ticker)[['SMA50','SMA200', 'RSI']]
    corr = df.corr()
    plot = sns.heatmap(df.corr(), annot=True)
    return plot

# Для получения графика индикаторов или корреляции достаточно просто вызвать соответствующие функции (например, draw_plots('SBER'))

In [38]:
draw_plots('SBER')